# Fietsdata processing
Verwerken van ruwe fietstellingdata naar data geaggregeerd per 24h, gekoppeld aan een wegdeel en verschillende databronnen samengenomen.

In [3]:
import psycopg2
try:
    postgres_pw = TokenLibrary.getSecret("redacted", "redacted", "redacted")
    conn = psycopg2.connect(dbname="rdt_dev",user="redacted",host="redacted",password=postgres_pw)
    conn.autocommit = True
    cur=conn.cursor()
    print("Database connected")
except:
    print("I am unable to connect to the database")

In [ ]:
# Wegenbestand
sql='''DROP TABLE IF EXISTS prd_fietsdata.doorfietsroute_cleaned;
CREATE TABLE prd_fietsdata.doorfietsroute_cleaned AS (
    select geom,wvk_id,jte_id_beg,jte_id_end,wegbehsrt,rpe_code,rijrichtng,stt_naam,stt_bron,wpsnaam
        ,gme_id,gme_naam,wegbehcode,wegbehnaam,geobron_id,geobron_nm,bronjaar,frc
    from stg_fietsdata.doorfietsroute
);'''
cur.execute(sql)

## PNH Measurement sites data prep
Adding geom and road_id

In [ ]:
# lon/lat to geom
sql='''DROP TABLE IF EXISTS int_fietsdata.pnh_fietstellingen_measurement_sites_geom;
CREATE TABLE int_fietsdata.pnh_fietstellingen_measurement_sites_geom AS (
    select measurepoint, ndwlocationid, source, bearing, s.name as streetname
        ,ST_Transform(ST_SetSRID(ST_MakePoint(longitude, latitude), 4326),28992) as geom
    FROM stg_fietsdata.pnh_fietstellingen_measurement_sites as s
);'''
cur.execute(sql)

In [ ]:
# geom to road_id, based on NDW dataset
sql='''DROP TABLE IF EXISTS int_fietsdata.pnh_fietstellingen_measurement_sites_geom_road_id;
CREATE TABLE int_fietsdata.pnh_fietstellingen_measurement_sites_geom_road_id AS (
    select g.measurepoint, g.ndwlocationid, g.source, g.bearing, g.streetname
        , abs(r.wvk_id) as road_id, r.geom, r.dist
    from int_fietsdata.pnh_fietstellingen_measurement_sites_geom as g
    CROSS JOIN LATERAL (
        SELECT roads.wvk_id, roads.geom, roads.geom <-> g.geom AS dist
        FROM prd_fietsdata.doorfietsroute_cleaned as roads
        ORDER BY dist
        LIMIT 1
    ) as r
    where r.dist < 50
);'''
cur.execute(sql)

## PNH Measured counts data prep
Aggregate per 24h

Create a table containing only road_id, datetime, (24h aggregate) counts

In [ ]:
# unix timestamp to postgres datetime
sql='''DROP TABLE IF EXISTS int_fietsdata.pnh_fietstellingen_measured_data_datetime;
CREATE TABLE int_fietsdata.pnh_fietstellingen_measured_data_datetime AS (
    select measurepoint,to_timestamp(starttime) as startTime,to_timestamp(endtime) as endTime,bothdirections,countto,countfrom,source
    FROM stg_fietsdata.pnh_fietstellingen_measured_data as d
);'''
cur.execute(sql)

In [ ]:
# aggregate per 24h
sql='''DROP TABLE IF EXISTS int_fietsdata.pnh_fietstellingen_measured_data_24h;
CREATE TABLE int_fietsdata.pnh_fietstellingen_measured_data_24h AS (
    select measurepoint,source,startTime::date as measurementdate,sum(bothdirections) as counts
    FROM int_fietsdata.pnh_fietstellingen_measured_data_datetime as d
    GROUP BY measurepoint,source,startTime::date
    ORDER BY startTime::date, measurepoint
);'''
cur.execute(sql)

In [ ]:
# using measurePoint and source, find correct road_id
sql='''DROP TABLE IF EXISTS prd_fietsdata.pnh_fietstellingen_measured_data_24h_road_id;
CREATE TABLE prd_fietsdata.pnh_fietstellingen_measured_data_24h_road_id AS (
    select d.measurepoint as beschrijving, d.measurementdate, d.counts::float as counts_BothWaysSummed
    , s.road_id, s.geom
    FROM int_fietsdata.pnh_fietstellingen_measured_data_24h as d
    -- join twice, as column d.source is mixed data
    inner join int_fietsdata.pnh_fietstellingen_measurement_sites_geom_road_id as s
    on d.source = s.source AND d.measurepoint = s.measurepoint
    union
    select d.measurepoint as beschrijving, d.measurementdate, d.counts::float as counts_BothWaysSummed
    , s2.road_id, s2.geom
    FROM int_fietsdata.pnh_fietstellingen_measured_data_24h as d
    inner join int_fietsdata.pnh_fietstellingen_measurement_sites_geom_road_id as s2
    on d.source = s2.source AND d.measurepoint = s2.ndwlocationid
);'''
cur.execute(sql)

## NDW fietsdata prep
make table with distinct measurement sites (based on name)
for each site:
- lon/lat to point
- point to road_id
- add geom based on road_id

for each datapoint/measurement:
- aggregate to 24h
- connect to road_id and geom using location name

In [ ]:
# distinct measurement sites
sql='''DROP TABLE IF EXISTS int_fietsdata.ndw_fietstellingen_measurement_sites_geom;
CREATE TABLE int_fietsdata.ndw_fietstellingen_measurement_sites_geom AS (
    select DISTINCT locatiecode, naam, breedtegraad, lengtegraad
        ,ST_Transform(ST_SetSRID(ST_MakePoint(lengtegraad::float,breedtegraad::float), 4326),28992) as geom
    FROM stg_fietsdata.fiets_data_export_schiphol as d
);'''
cur.execute(sql)

In [ ]:
# geom to road_id, based on NDW dataset
sql='''DROP TABLE IF EXISTS int_fietsdata.ndw_fietstellingen_measurement_sites_geom_roadid;
CREATE TABLE int_fietsdata.ndw_fietstellingen_measurement_sites_geom_roadid AS (
    select g.locatiecode, g.naam
        , abs(r.wvk_id) as road_id, r.geom, r.dist
    from int_fietsdata.ndw_fietstellingen_measurement_sites_geom as g
    CROSS JOIN LATERAL (
        SELECT roads.wvk_id, roads.geom, roads.geom <-> g.geom AS dist
        FROM prd_fietsdata.doorfietsroute_cleaned as roads
        ORDER BY dist
        LIMIT 1
    ) as r
    where r.dist < 50
);'''
cur.execute(sql)

In [ ]:
# unix timestamp to postgres datetime
sql='''DROP TABLE IF EXISTS int_fietsdata.ndw_fietstellingen_measured_data_datetime;
CREATE TABLE int_fietsdata.ndw_fietstellingen_measured_data_datetime AS (
    select locatiecode,naam,to_timestamp(begintijd,'YYYY-MM-DD HH24:MI:SS') as startTime, intensiteit_beide_richtingen
    FROM stg_fietsdata.fiets_data_export_schiphol as d
);'''
cur.execute(sql)

In [ ]:
# aggregate per 24h
sql='''DROP TABLE IF EXISTS int_fietsdata.ndw_fietstellingen_measured_data_24h;
CREATE TABLE int_fietsdata.ndw_fietstellingen_measured_data_24h AS (
    select locatiecode,naam,startTime::date as measurementdate,sum(intensiteit_beide_richtingen::float) as counts
    FROM int_fietsdata.ndw_fietstellingen_measured_data_datetime as d
    WHERE intensiteit_beide_richtingen!=''
    GROUP BY locatiecode,naam,startTime::date
    ORDER BY startTime::date, locatiecode
);'''
cur.execute(sql)

In [ ]:
# using measurePoint and source, find correct road_id
sql='''DROP TABLE IF EXISTS prd_fietsdata.ndw_fietstellingen_measured_data_24h_road_id;
CREATE TABLE prd_fietsdata.ndw_fietstellingen_measured_data_24h_road_id AS (
    select d.locatiecode as beschrijving, d.measurementdate, d.counts as counts_BothWaysSummed
    , s.road_id, s.geom
    FROM int_fietsdata.ndw_fietstellingen_measured_data_24h as d
    inner join int_fietsdata.ndw_fietstellingen_measurement_sites_geom_roadid as s
    on d.locatiecode = s.locatiecode
);'''
cur.execute(sql)

## Interpolation of datapoints

In [7]:
%run RDT_FietsdataInterpolate

## Fietsdata combinatie in 1 tabel

In [ ]:
# using measurePoint and source, find correct road_id
sql='''DROP TABLE IF EXISTS int_fietsdata.fietstellingen_24h_road_id;
CREATE TABLE int_fietsdata.fietstellingen_24h_road_id AS (
    SELECT 'ndw' as source
        , n.beschrijving, n.measurementdate, n.counts_BothWaysSummed, n.road_id
        ,n.geom
    from prd_fietsdata.ndw_fietstellingen_measured_data_24h_road_id as n
    union
    SELECT 'pnh' as source
        , p.beschrijving, p.measurementdate, p.counts_BothWaysSummed, p.road_id
        ,p.geom
    from prd_fietsdata.pnh_fietstellingen_measured_data_24h_road_id as p
    union
    SELECT *
    from prd_fietsdata.fietstellingen_24h_road_id_interpolated as p
);

DROP TABLE IF EXISTS prd_fietsdata.fietstellingen_24h_road_id_4326;
CREATE TABLE prd_fietsdata.fietstellingen_24h_road_id_4326 AS (
    SELECT source, beschrijving, measurementdate, counts_BothWaysSummed, road_id
        ,ST_Transform(geom,4326) as geom
    from int_fietsdata.fietstellingen_24h_road_id
);'''
cur.execute(sql)